In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import random
import datetime
from scipy.stats import norm, gamma, expon, lognorm, poisson
import ipywidgets as widgets
from IPython.display import display, clear_output
from IPython.display import HTML
import plotly.graph_objects as go
from plotly.subplots import make_subplots
pd.set_option('display.max_columns', None)

# Set a random seed for reproducibility
random.seed(42)

# Generate random data
item_ids = np.random.randint(1, 1001, size=10000)
period_seqs = np.random.randint(1, 13, size=10000)
brand_names = [f'Brand-{random.randint(1, 10)}' for _ in range(10000)]

data = {
    'item_id': item_ids,
    'period_seq': period_seqs,
    'item_group_code': [f'Group-{random.randint(1, 5)}' for _ in range(10000)],
    'country_code': ['SingleCountry' for _ in range(10000)],
    'periodicity': ['Weekly' for _ in range(10000)],
    'market_depth': [random.choice(['Shallow', 'Medium', 'Deep']) for _ in range(10000)],
    'sales_units': np.round(poisson.rvs(500, size=10000), 2),  # Sales units with Poisson distribution
    'sales_value': np.round(lognorm.rvs(2, scale=1000, size=10000), 2),  # Sales value with log-normal distribution
    'sales_units_tpr': np.round(gamma.rvs(2, scale=0.5, size=10000), 2),  # Sales units tpr with gamma distribution
    'sales_value_tpr': np.round(expon.rvs(scale=0.7, size=10000), 2),  # Sales value tpr with exponential distribution
    'salesunits_increment': [str(random.uniform(0, 0.5)) for _ in range(10000)],
    'salesvalue_increment': [str(random.uniform(0, 0.2)) for _ in range(10000)],
    'promotion_cost': np.round(norm.rvs(300, 100, size=10000), 2),  # Promotion cost with normal distribution
    'is_tradebrand': [random.choice([True, False]) for _ in range(10000)],
    'start_date': [datetime.datetime(2022, random.randint(1, 12), random.randint(1, 28)) for _ in range(10000)],
    'end_date': [datetime.datetime(2022, random.randint(1, 12), random.randint(1, 28)) for _ in range(10000)],
    'brand_name': brand_names,
    'item_name': [f'Item-{i}' for i in range(1, 10001)],
    'outlet_id': np.arange(1, 10001),
    'basechannel_id': np.random.randint(1, 6, size=10000),
    'channel_id': np.random.randint(101, 106, size=10000),
    'retailer_id': np.arange(10001, 20001)
}

# Create a DataFrame
df = pd.DataFrame(data)

# Ensure unique combinations of 'item_id' and 'period_seq'
df.drop_duplicates(subset=['item_id', 'period_seq'], keep='first', inplace=True)

# Reset the index after dropping duplicates
df.reset_index(drop=True, inplace=True)


In [3]:
def convert_columns_to_float(df, columns):
    for column in columns:
        df[column] = df[column].astype(float)
    return df


columns_to_convert = ['sales_units_tpr', 'sales_value_tpr', 'salesunits_increment', 'salesvalue_increment', 'promotion_cost']
df_goups = convert_columns_to_float(df, columns_to_convert)

In [4]:
# Group the data by brand_name and calculate the sum of sales_units for each brand
brand_sales_units = df_goups.groupby('brand_name')['sales_units'].sum().reset_index()

# Sort the brands by total sales units in descending order
brand_sales_units = brand_sales_units.sort_values(by='sales_units', ascending=False)

# Select the top 5 brands
top_5_brands_units = brand_sales_units.head(5)

# Create a bar chart to visualize the top 5 brands with the highest sales units
fig_units = px.bar(top_5_brands_units, x='brand_name', y='sales_units', title='Top 5 Brands by Sales Units')
fig_units.update_xaxes(title_text='Brand Name')
fig_units.update_yaxes(title_text='Total Sales Units')
fig_units.update_traces(text=top_5_brands_units['sales_units'], textposition='inside', texttemplate='%{y}')

# Initialize the output area
output_sales_units = widgets.Output()

# Define a function to update the plot and display the top N brands
def update_top_brands_units(change):
    N = top_brands_slider_units.value  # Number of top brands to display

    top_N_brands_units = brand_sales_units.head(N)

    # Create a bar chart to visualize the top N brands with the highest sales units
    fig_units = px.bar(top_N_brands_units, x='brand_name', y='sales_units', title=f'Top {N} Brands by Sales Units')
    fig_units.update_xaxes(title_text='Brand Name')
    fig_units.update_yaxes(title_text='Total Sales Units')
    fig_units.update_traces(text=top_N_brands_units['sales_units'], textposition='inside', texttemplate='%{y}')

    with output_sales_units:
        clear_output(wait=True)
        display(fig_units)

# Create a slider for selecting the number of top brands
top_brands_slider_units = widgets.IntSlider(
    value=5, min=1, max=len(brand_sales_units), step=1, description='Top N Brands (Units):', continuous_update=False
)

# Define an event handler for the slider
top_brands_slider_units.observe(update_top_brands_units, names='value')  

# Display the initial top 5 brands plot and the slider
with output_sales_units:
    display(fig_units)

# Display the slider and the output area
display(top_brands_slider_units)
display(output_sales_units)

IntSlider(value=5, continuous_update=False, description='Top N Brands (Units):', max=10, min=1)

Output()

In [6]:
# Group the data by brand_name and calculate the sum of sales_value for each brand
brand_sales_value = df_goups.groupby('brand_name')['sales_value'].sum().reset_index()

# Sort the brands by total sales value in descending order
brand_sales_value = brand_sales_value.sort_values(by='sales_value', ascending=False)

# Create an initial bar chart for the top 5 brands with default number of bins
N = 5
top_N_brands_value = brand_sales_value.head(N)
fig_value = px.bar(top_N_brands_value, x='brand_name', y='sales_value', title=f'Top {N} Brands by Sales Value')
fig_value.update_xaxes(title_text='Brand Name')
fig_value.update_yaxes(title_text='Total Sales Value')
fig_value.update_traces(text=top_N_brands_value['sales_value'], textposition='inside', texttemplate='%{y}')

# Initialize the output area
output_value_brand = widgets.Output()

# Define a function to update the plot and display the top N brands
def update_top_brands_value(change):
    N = top_brands_slider_value.value  # Number of top brands to display

    top_N_brands_value = brand_sales_value.head(N)

    # Create a bar chart with the updated number of bins
    fig_value = px.bar(top_N_brands_value, x='brand_name', y='sales_value', title=f'Top {N} Brands by Sales Value')
    fig_value.update_xaxes(title_text='Brand Name')
    fig_value.update_yaxes(title_text='Total Sales Value')
    fig_value.update_traces(text=top_N_brands_value['sales_value'], textposition='inside', texttemplate='%{y}')

    with output_value_brand:
        clear_output(wait=True)
        display(fig_value)

# Create a slider for selecting the number of top brands
top_brands_slider_value = widgets.IntSlider(
    value=5, min=1, max=len(brand_sales_value), step=1, description='Top N Brands (Value):', continuous_update=False
)

# Define an event handler for the slider
top_brands_slider_value.observe(update_top_brands_value, names='value')

# Display the initial top 5 brands plot and the slider
with output_value_brand:
    display(fig_value)

# Display the slider and the output area
display(top_brands_slider_value)
display(output_value_brand)

IntSlider(value=5, continuous_update=False, description='Top N Brands (Value):', max=10, min=1)

Output()

In [ ]:
# Function to format numbers as "M" for millions and "K" for thousands
def format_number(number):
    if number >= 1e6:
        return f'{number / 1e6:.2f}M'
    elif number >= 1e3:
        return f'{number / 1e3:.2f}K'
    else:
        return str(number)

# Create a list of unique brand names for sales units
brand_names_units = df_goups['brand_name'].unique()

# Initialize the output area for sales units
output_units = widgets.Output()

# Define a function to update the sales units section and display the total number of items, total sales units, and the top N items by sales units
def update_top_items_units(change):
    selected_brand = brand_dropdown_units.value
    N = top_items_slider_units.value  # Number of top items to display
    filtered_df_units = df_goups[df_goups['brand_name'] == selected_brand]

    # Calculate the total number of items sold and total sales units for the selected brand
    total_items_sold_units = filtered_df_units['item_id'].nunique()
    total_sales_units = filtered_df_units['sales_units'].sum()

    # Calculate the total sales units for each item
    item_sales_units = filtered_df_units.groupby('item_name')['sales_units'].sum().reset_index()

    # Sort the items by total sales units in descending order
    sorted_items_units = item_sales_units.sort_values(by='sales_units', ascending=False)

    # Create a data frame for the top N items by sales units
    top_items_units = sorted_items_units.head(N)

    # Define a custom sorting order for the top items based on sales units
    custom_sort_units = top_items_units['item_name'].values.tolist()

    # Create a bar chart for the top N items by sales units with the custom sorting order
    fig_units = px.bar(top_items_units, x='item_name', y='sales_units', title=f'Top {N} Items by Sales Units for {selected_brand}', category_orders={"item_name": custom_sort_units})
    fig_units.update_xaxes(title_text='Item Name')
    fig_units.update_yaxes(title_text='Sales Units')
    fig_units.update_traces(text=top_items_units['sales_units'], textposition='inside', texttemplate='%{y}')

    with output_units:
        clear_output(wait=True)
        print(f'Total Number of Items Sold for {selected_brand}: {total_items_sold_units}')
        print(f'Total Sales Units for {selected_brand}: {format_number(total_sales_units)}')
        print(f'Top {N} Items by Sales Units for {selected_brand} (in custom sorting order):')
        display(top_items_units)  # Now display the top items directly for better formatting
        print(f'Total Sales Units for top {N} items for {selected_brand}: {format_number(top_items_units["sales_units"].sum())}')
        display(fig_units)

# Create a dropdown widget for selecting the brand name for sales units
brand_dropdown_units = widgets.Dropdown(
    options=brand_names_units,
    description='Select Brand (Units):'
)

# Create a slider for selecting the number of top items by sales units
top_items_slider_units = widgets.IntSlider(
    value=5, min=1, max=25, step=1, description='Top N Items (Units):', continuous_update=False
)

# Define an event handler for the dropdown widget for sales units
brand_dropdown_units.observe(update_top_items_units, names='value')
top_items_slider_units.observe(update_top_items_units, names='value')

# Display the dropdown widget, slider, and output area for sales units
display(brand_dropdown_units)
display(top_items_slider_units)
display(output_units)

Dropdown(description='Select Brand (Units):', options=('Brand-2', 'Brand-1', 'Brand-5', 'Brand-4', 'Brand-3', …

IntSlider(value=5, continuous_update=False, description='Top N Items (Units):', max=25, min=1)

Output()

In [ ]:
# Function to format numbers as "M" for millions and "K" for thousands
def format_number(number):
    if number >= 1e6:
        return f'{number / 1e6:.2f}M'
    elif number >= 1e3:
        return f'{number / 1e3:.2f}K'
    else:
        return str(number)

# Create a list of unique brand names for sales value
brand_names_value = df_goups['brand_name'].unique()

# Initialize the output area for sales value
output_value = widgets.Output()

# Define a function to update the sales value section and display the total number of items, total sales value, and the top N items by sales value
def update_top_items_value(change):
    selected_brand = brand_dropdown_value.value
    N = top_items_slider_value.value  # Number of top items to display
    filtered_df_value = df_goups[df_goups['brand_name'] == selected_brand]

    # Calculate the total number of items sold and total sales value for the selected brand
    total_items_sold_value = filtered_df_value['item_id'].nunique()
    total_sales_value = filtered_df_value['sales_value'].sum()

    # Calculate the total sales value for each item
    item_sales_value = filtered_df_value.groupby('item_name')['sales_value'].sum().reset_index()

    # Sort the items by total sales value in descending order
    sorted_items_value = item_sales_value.sort_values(by='sales_value', ascending=False)

    # Create a data frame for the top N items by sales value
    top_items_value = sorted_items_value.head(N)

    # Define a custom sorting order for the top items based on sales value
    custom_sort_value = top_items_value['item_name'].values.tolist()

    # Create a bar chart for the top N items by sales value with the custom sorting order
    fig_value = px.bar(top_items_value, x='item_name', y='sales_value', title=f'Top {N} Items by Sales Value for {selected_brand}', category_orders={"item_name": custom_sort_value})
    fig_value.update_xaxes(title_text='Item Name')
    fig_value.update_yaxes(title_text='Sales Value')
    fig_value.update_traces(text=top_items_value['sales_value'], textposition='inside', texttemplate='%{y}')

    with output_value:
        clear_output(wait=True)
        print(f'Total Number of Items Sold for {selected_brand}: {total_items_sold_value}')
        print(f'Total Sales Value for {selected_brand}: {format_number(total_sales_value)}')
        print(f'Top {N} Items by Sales Value for {selected_brand} (in custom sorting order):')
        display(top_items_value)  # Now display the top items directly for better formatting
        print(f'Total Sales Value for top {N} items for {selected_brand}: {format_number(top_items_value["sales_value"].sum())}')
        display(fig_value)

# Create a dropdown widget for selecting the brand name for sales value
brand_dropdown_value = widgets.Dropdown(
    options=brand_names_value,
    description='Select Brand (Value):'
)

# Create a slider for selecting the number of top items by sales value
top_items_slider_value = widgets.IntSlider(
    value=5, min=1, max=25, step=1, description='Top N Items (Value):', continuous_update=False
)

# Define an event handler for the dropdown widget for sales value
brand_dropdown_value.observe(update_top_items_value, names='value')
top_items_slider_value.observe(update_top_items_value, names='value')

# Display the dropdown widget, slider, and output area for sales value
display(brand_dropdown_value)
display(top_items_slider_value)
display(output_value)

In [8]:
# Function to format numbers as "M" for millions and "K" for thousands
def format_number(number):
    if number >= 1e6:
        return f'{number / 1e6:.2f}M'
    elif number >= 1e3:
        return f'{number / 1e3:.2f}K'
    else:
        return str(number)

# Create a list of unique brand names for sales units and sales value
brand_names = df_goups['brand_name'].unique()

# Initialize the output area for sales units and sales value
output_units_value = widgets.Output()

# Define a function to update the sales units and sales value sections and display them together for the selected brand
def update_top_items_units_value(change):
    selected_brand = brand_dropdown_units_value.value
    N = top_items_slider_units_value.value  # Number of top items to display
    filtered_df_units_value = df_goups[df_goups['brand_name'] == selected_brand]

    # Calculate the total number of items sold, total sales units, and total sales value for the selected brand
    total_items_sold_units_value = filtered_df_units_value['item_id'].nunique()
    total_sales_units = filtered_df_units_value['sales_units'].sum()
    total_sales_value = filtered_df_units_value['sales_value'].sum()

    # Calculate the total sales units and sales value for each item
    item_sales_units = filtered_df_units_value.groupby('item_name')['sales_units'].sum().reset_index()
    item_sales_value = filtered_df_units_value.groupby('item_name')['sales_value'].sum().reset_index()

    # Sort the items by total sales units and total sales value in descending order
    sorted_items_units = item_sales_units.sort_values(by='sales_units', ascending=False)
    sorted_items_value = item_sales_value.sort_values(by='sales_value', ascending=False)

    # Create data frames for the top N items by sales units and sales value
    top_items_units = sorted_items_units.head(N)
    top_items_value = sorted_items_value.head(N)

    # Define custom sorting orders for the top items based on sales units and sales value
    custom_sort_units = top_items_units['item_name'].values.tolist()
    custom_sort_value = top_items_value['item_name'].values.tolist()

    # Create bar charts for the top N items by sales units and sales value with custom sorting orders
    fig_units = px.bar(top_items_units, x='item_name', y='sales_units', title=f'Top {N} Items by Sales Units for {selected_brand}', category_orders={"item_name": custom_sort_units})
    fig_units.update_xaxes(title_text='Item Name')
    fig_units.update_yaxes(title_text='Sales Units')

    # Add labels to the top of the bins for sales units
    fig_units.update_traces(text=top_items_units['sales_units'], textposition='inside', texttemplate='%{y}')

    fig_value = px.bar(top_items_value, x='item_name', y='sales_value', title=f'Top {N} Items by Sales Value for {selected_brand}', category_orders={"item_name": custom_sort_value})
    fig_value.update_xaxes(title_text='Item Name')
    fig_value.update_yaxes(title_text='Sales Value')

    # Add labels to the top of the bins for sales value
    fig_value.update_traces(text=top_items_value['sales_value'], textposition='inside', texttemplate='%{y}')

    with output_units_value:
        clear_output(wait=True)
        display(HTML(f'<h4>Total Number of Items Sold for {selected_brand}: {total_items_sold_units_value}</h4>'))
        display(HTML(f'<h4>Total Sales Units for {selected_brand}: {format_number(total_sales_units)}</h4>'))
        display(HTML(f'<h4>Total Sales Value for {selected_brand}: ${format_number(total_sales_value)}</h4>'))
        display(HTML(f'<h4>Top {N} Items by Sales Units for {selected_brand} (in custom sorting order):</h4>'))
        display(top_items_units)  # Now display the top items directly for better formatting
        display(HTML(f'<h4>Total Sales Units for top {N} items for {selected_brand}: {format_number(top_items_units["sales_units"].sum())}</h4>'))
        display(fig_units)
        display(fig_value)
        display(HTML(f'<h4>Total Sales Value for top {N} items for {selected_brand}: ${format_number(top_items_value["sales_value"].sum())}</h4>'))
        display(top_items_value)

# Create a dropdown widget for selecting the brand name for sales units and sales value
brand_dropdown_units_value = widgets.Dropdown(
    options=brand_names,
    description='Select Brand:'
)

# Create a slider for selecting the number of top items by sales units and sales value
top_items_slider_units_value = widgets.IntSlider(
    value=5, min=1, max=25, step=1, description='Top N Items:', continuous_update=False
)

# Define event handlers for the dropdown widget and slider
brand_dropdown_units_value.observe(update_top_items_units_value, names='value')
top_items_slider_units_value.observe(update_top_items_units_value, names='value')

# Display the dropdown widget, slider, and output area for sales units and sales value
display(brand_dropdown_units_value)
display(top_items_slider_units_value)
display(output_units_value)

# Initial display
with output_units_value:
    update_top_items_units_value(None)

Dropdown(description='Select Brand:', options=('Brand-2', 'Brand-1', 'Brand-5', 'Brand-4', 'Brand-3', 'Brand-9…

IntSlider(value=5, continuous_update=False, description='Top N Items:', max=25, min=1)

Output()

In [7]:
brand_sales = df_goups.groupby('brand_name').agg({
    'sales_units': 'sum',
    'sales_units_tpr': 'sum'
}).reset_index()

time_series_data = df_goups.groupby('start_date').agg({
    'sales_units': 'sum',
    'sales_units_tpr': 'sum'
}).reset_index()

time_series_data = time_series_data.melt(id_vars='start_date', var_name='Metric', value_name='Total Sales Units')

fig = px.line(time_series_data, x='start_date', y='Total Sales Units', color='Metric',
              labels={'start_date': 'Start Date', 'Total Sales Units': 'Total Sales Units'},
              title='Time Series of Sales Units and Sales Units TPR')
fig.update_xaxes(title_text='Start Date')
fig.update_yaxes(title_text='Total Sales Units')
fig.show()
